<a href="https://colab.research.google.com/github/dtabuena/Patch_Ephys/blob/main/Refactor/Analyze_Recs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
'Get Standard Modules'

import pandas as pd
import numpy as np
import matplotlib
import pyabf
import matplotlib.pyplot as plt
import scipy
from scipy import stats
import os
from scipy.signal import butter,filtfilt
from tqdm import tqdm
import warnings
from IPython.display import clear_output
from datetime import datetime
import sys
import warnings
import shutil
import urllib.request
import sklearn as skl
import sklearn.mixture as skl_mixture
import seaborn as sns
# warnings.filterwarnings('ignore')
# np.set_printoptions(threshold=sys.maxsize)
# clear_output(wait=False)


!pip install openpyxl
!pip install XlsxWriter

In [19]:
'''Get Repositories'''
"run dtabuena's ephys notebooks"
to_import = [
            # 'ABF_Quality_Control.ipynb',
            'Basic_Ephys.ipynb',
            # 'Simple_ABF_tools.ipynb',
            # 'fun_math.ipynb',
            # 'importing_abfs_from_dropbox.ipynb',
            'QC_recoding_dataframe.ipynb',
            'input_resistance_analyzer.ipynb',
            'gain_analyzer.ipynb',
            'latencey_analyzer.ipynb',
            'IV_analyzer.ipynb',
            'rmp_analyzer.ipynb',
            'membrane_analyzer.ipynb',
            'Ephys_wrapper.ipynb',
            'rheobase_analyzer.ipynb',
            'init_func_arg_dicts.ipynb',
            ]



parent_folder = r"C:\Users\dennis.tabuena\Gladstone Dropbox\Dennis Tabuena\4_Methods\_Patch_Refactor"
os.chdir(parent_folder)

code_dir = 'git_code'
# shutil.rmtree(code_dir)
os.makedirs(code_dir,exist_ok=True)
os.chdir(code_dir)

for f in to_import:
    try:
        git_link = 'https://raw.githubusercontent.com/dtabuena/Patch_Ephys/main/'+f
        # print(git_link)
        urllib.request.urlretrieve(git_link,f)
        %run $f
        print('loaded:',f)
    except:
        print('FAILED:',f)


import urllib
# response = urllib.request.urlretrieve('https://raw.githubusercontent.com/dtabuena/Resources/main/Matplotlib_Config/Load_FS6.py','Load_FS6.py')
# %run Load_FS6.py

basic_ephys loaded succesfully
loaded: Basic_Ephys.ipynb
loaded: QC_recoding_dataframe.ipynb
loaded: input_resistance_analyzer.ipynb
loaded: gain_analyzer.ipynb
loaded: latencey_analyzer.ipynb
loaded: IV_analyzer.ipynb
loaded: rmp_analyzer.ipynb
loaded: membrane_analyzer.ipynb
loaded: Ephys_wrapper.ipynb
loaded: rheobase_analyzer.ipynb
loaded: init_func_arg_dicts.ipynb


In [20]:
def init_func_arg_dicts():
    func_dict = {}
    arg_dict = {}

    # Spike arg defentitions
    spike_args_gain={'spike_thresh':10, 'high_dv_thresh': 20,'low_dv_thresh': -5,'window_ms': 3}
    spike_args_rheo ={'spike_thresh':15, 'high_dv_thresh': 30,'low_dv_thresh': -15,'window_ms': 2}



    func_dict['VC - 3min GapFree']= rmp_analyzer
    arg_dict['VC - 3min GapFree'] = [True] # [to_plot?]

    func_dict['I0 - 3min GapFree']= rmp_analyzer
    arg_dict['I0 - 3min GapFree'] = [True] # [to_plot?]

    func_dict['IC - Rheobase']= rheobase_analyzer
    arg_dict['IC - Rheobase'] = [spike_args_rheo, True, False, False]  # [spike_args, to_plot, verbose, force_singlespike]

    func_dict['IC - Gain - D10pA']= gain_analyzer
    arg_dict['IC - Gain - D10pA']= [spike_args_gain, 1, 4, .7,[-60,-80]]  # [spike_args, to_plot [0:2], max_fit_steps, rel_slope_cut, Vh_hilo]

    func_dict['IC - Gain - D20pA']= gain_analyzer
    arg_dict['IC - Gain - D20pA']= [spike_args_gain, 1, 4, .7,[-60,-80]]  # [spike_args, to_plot [0:2], max_fit_steps, rel_slope_cut, Vh_hilo]

    func_dict['IC - Gain - D25pA']= gain_analyzer
    arg_dict['IC - Gain - D25pA']= [spike_args_gain, 1, 4, .7,[-60,-80]]  # [spike_args, to_plot [0:2], max_fit_steps, rel_slope_cut, Vh_hilo]

    func_dict['IC - Gain - D50pA']= func_dict['IC - Gain - D20pA']
    arg_dict['IC - Gain - D50pA']= arg_dict['IC - Gain - D20pA']

    func_dict['VC - MemTest-10ms-160ms']= membrane_analyzer
    arg_dict['VC - MemTest-10ms-160ms']= [True, False, ['Ra', 'Rm', 'Cm', 'tau',	'Cmq',	'Cmf',	'Cmqf', 'Cm_pc']]  # [to_plot, verbose]

    func_dict['IC - Latentcy 800pA-1s']= latencey_analyzer
    arg_dict['IC - Latentcy 800pA-1s']= [spike_args_gain, True]  # [spike_args, to_plot]

    func_dict['IC - R input']= input_resistance_analyzer
    arg_dict['IC - R input']= [[-30, 10] ,True]  # [dVm_limits, to_plot]

    # func_dict['VC - Multi IV - 150ms'] = IV_analyzer_v2
    # arg_dict['VC - Multi IV - 150ms']= [{'IV_Early':(4.5, 30),'IV_Steady_State':(70,80)} ,[False, True]]  # [measure_windows, to_plot]

    func_dict['VC - Multi IV - 150ms'] = IV_analyzer_v3
    arg_dict['VC - Multi IV - 150ms']= [[.2, 3],[40,50],True,]  # [Na_window,K_window, to_plot]

    return func_dict, arg_dict



In [21]:
##### Setup Protocol List
VC_prot = ['VC - MemTest-10ms-160ms',
           'VC - Multi IV - 150ms',]
IC_prot = ['IC - Gain - D10pA',
           'IC - Gain - D20pA',
           'IC - Gain - D50pA',
           'IC - Rheobase',
           'IC - R input',
           'IC - Latentcy 800pA-1s'
           'VC - 3min GapFree',
           'I0 - 3min GapFree']


# dataset = {'data_name': 'test_data',
#            'data_source': r"C:\Users\dennis.tabuena\Gladstone Dropbox\Dennis Tabuena\4_Methods\_Patch_Refactor\test_data",
#            'file_naming_scheme': ['Rec_date','geno','sex','age','orientation','slice','cell','type'],
#            }

dataset = {'data_name': 'test_data',
           'data_source': r"C:\Users\dennis.tabuena\Gladstone Dropbox\Dennis Tabuena\4_Methods\_Patch_Refactor\test_data",
           'file_naming_scheme': ['year','month','day'],
           }

analysis_dir =os.path.join(parent_folder,dataset['data_name'])
os.makedirs(analysis_dir,exist_ok=True)
os.chdir(analysis_dir)
print(os.getcwd())


C:\Users\dennis.tabuena\Gladstone Dropbox\Dennis Tabuena\4_Methods\_Patch_Refactor\test_data


In [22]:
"""
Scripted Run
(unwraped 'ephys wrapper')
"""
# init Output
results = {}


'''Build Basic DF'''
abf_recordings_df, protocol_set = catalogue_recs(dataset['data_source'],dataset['file_naming_scheme'])
results['abf_recordings_df'] = abf_recordings_df
results['protocol_set'] = protocol_set

'''# Protocol Look Up Table'''
csv_name = cell_prot_lut(abf_recordings_df,protocol_set,csv_name=dataset['data_name']+'_Recording_LookUp')
results['prot_lut'] = csv_name


func_dict, arg_dict = init_func_arg_dicts()

'''Protocol Look Up Table'''
abf_recordings_df, problem_recs = analysis_iterator(abf_recordings_df,func_dict,arg_dict,verbose=True)
clear_output()
display(abf_recordings_df.head(3))
abf_recordings_df.to_csv('abf_recordings_df.csv')

'''Sort Cells'''
cell_df = cell_sorting(abf_recordings_df)
results['cell_df'] = cell_df
display(cell_df.head(3))
cell_df.to_csv('cell_df.csv')

NameError: name 'catalogue_recs' is not defined

In [ ]:

fast_results = ephys_wrapper_local(dataset,VC_prot,IC_prot,strat_cols=['day'],size_col='Cmq_128.0')

In [ ]:
print(os.getcwd().replace('\\','/'))

In [ ]:
# fast_results = substitute_gain_rheobase(fast_results,
#                                        strat_cols=['Genotype','Age_Bin','Cell_Type'],
#                                        age_bin_dict=age_bin_dict)


In [ ]:
f = fast_results

In [ ]:

single_val_strat_groups = ['ap_amplitutude',
                            'Rmp_mV',
                            'Cmq_128.0',
                            'Ra_128.0',
                            'Rm_128.0',
                            'Gain_(HzpA)',
                            'inact_current_pA',
                            'Rheobase',
                            'AP_thresh_US',
                            'fast_after_hyperpol',
                            'Spike_latency_(ms)',
                            'Input_Resistance_MO',
                            ]


manual_exclusions = ['']
exclusion_overide = ['']
file_naming_scheme = dataset['file_naming_scheme']
strat_df_dict = fast_results['strat_df_dict'].copy()
strat_df_dict = {k:strat_df_dict[k] for k in my_order}
filtered_dict, fail_dict = final_qc(strat_df_dict,file_naming_scheme,qc_Rmp=-35,qc_AP_amp=20,qc_RR=.20,qc_Ra=np.inf,manual_exclusions=manual_exclusions,exclusion_overide=exclusion_overide)
alt_strat_dict = restratify_results(filtered_dict,file_naming_scheme,single_val_strat_groups)

resp_curve_list = ['IV_Early_(V_stim)','IV_Early_(I_peak)','IV_Steady_State_(I_mean)','Stim_Levels_(pA)','Spike_Counts']
response_curve_data = stratify_response_curve(filtered_dict,resp_curve_list,strat_list=[''])
alt_strat_dict.update(response_curve_data)
write_strat_dfs_local(alt_strat_dict, dataset['data_name']+'_results_stratified_alternate')
print(os.getcwd())


In [ ]:
feature_list = ['Cm_pc_10.0',
             'Cmq_160.0',
             'Input_Resistance_MO',
             'Rm_160.0',
             'Rmp_mV',
             'Gain_(HzpA)',
             'Spike_latency_(ms)',
             'Rheobase',
             'inact_current_pA',
             'ap_amplitutude',
             'fast_after_hyperpol',
             'sAHP',
             'AP_thresh_US',
             'ap50_width_ms']



In [ ]:
def generate_prism_csv(data_df,max_replicates=None):
    data_type = data_df.columns[0]

    # Step 1: Group data by 'Age_Bin' and 'Genotype' and collect values
    grouped = data_df.groupby(['Age_Bin', 'Genotype'])[data_type].apply(list).reset_index()

    # Step 2: Determine maximum number of replicates based on the size of groups
    if max_replicates is None:
        max_replicates = grouped[data_type].str.len().max()

    # Step 3: Prepare a DataFrame for the multi-level index
    results = []

    # Replicate numbering function (retained as per your request)
    def rep_num(i):
        if i == 0:
            return ""
        else:
            return f'{i + 1:03}'  # Returns '001', '002', etc.

    # Step 4: Generate all possible rows, padding with None where necessary
    for _, row in grouped.iterrows():
        age_bin = row['Age_Bin']
        geno_type = row['Genotype']
        values = row[data_type]

        # Ensure every group has max_replicates, padding missing values with None
        for i in range(max_replicates):
            if i < len(values):
                # If value exists, use it
                results.append({'Age_Bin': age_bin, 'Genotype': geno_type, 'Replicate': rep_num(i), 'Value': values[i]})
            else:
                # If no value, add a row with None
                results.append({'Age_Bin': age_bin, 'Genotype': geno_type, 'Replicate': rep_num(i), 'Value': None})

    # Step 5: Create a DataFrame from the results
    final_df = pd.DataFrame(results)

    # Step 6: Pivot the DataFrame to have Age_Bin as index and multi-level columns, ensuring all replicates exist
    pivot_df = final_df.pivot_table(index='Age_Bin', columns=['Genotype', 'Replicate'], values='Value', aggfunc='first')

    # Step 7: Ensure all replicate columns exist up to max_replicates, even if some are missing in the data
    all_replicates = [rep_num(i) for i in range(max_replicates)]
    pivot_df = pivot_df.reindex(columns=pd.MultiIndex.from_product([pivot_df.columns.levels[0], all_replicates]), fill_value=None)

    # Step 8: Reset index for easier display
    pivot_df.reset_index(inplace=True)

    return pivot_df, max_replicates



def flatten_multiindex(df):
    df.columns = [''.join(map(str, col)).strip() for col in df.columns.values]
    return df

